In [85]:
import urllib.request as req
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from datetime import datetime
import multiprocessing
from multiprocessing import Pool

In [79]:
def main_cr(href):
    mainres = req.urlopen("https://finance.naver.com/" + href)
    mainsoup = BeautifulSoup(mainres,"lxml")
    mainsoup = mainsoup.find("div",class_="articleCont")
    for i in mainsoup.find_all("div"):
        i.decompose()
    return mainsoup.text

In [80]:
def ftn(date,page):
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page={}'.format(date,str(page))
    res = req.urlopen(url)
    soup = BeautifulSoup(res,"lxml")

    tmp_soup = soup.find("ul",class_="newsList").find_all("a")
    title = [x.get("title") for x in tmp_soup]
    href = [x.get("href") for x in tmp_soup]
    press = [x.text.strip() for x in soup.find_all("span",class_ = "press")]
    main = [main_cr(x) for x in href]
    return pd.DataFrame({"Title":title,"Href":href,"Date":date,"Press":press, "Main" : main})

In [83]:
def F_ftn(date):
    res = req.urlopen("https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page=1".format(date))
    soup = BeautifulSoup(res,"lxml")
    
    if len(soup.find("ul",class_="newsList").find_all("a")) == 0:
        return pd.DataFrame({"Title":[],"Href":[],"Date":[],"Press":[], "Main":[]})
    elif soup.find("td",class_ = "pgRR").text == "":
        return ftn(date,1)
    else:
        Last_page = int(soup.find("td",class_ = "pgRR").find("a").get("href").split("=")[-1])
        return pd.concat([ftn(date,page) for page in range(1,Last_page+1)],ignore_index=True)

In [84]:
F_ftn('20210723')

,Title,Href,Date,Press,Main
0,거래정지 삼성重…유증에 거는 희망,/news/news_read.nhn?article_id=0004942281&offi...,20210723,아시아경제,\n5대1 무상감자 돌입11월 예상 유증까지 마치면 부채비율 낮출 것으로 기대[이미...
1,"현대모비스, 10년 당긴 2040년 'RE100' 약속",/news/news_read.nhn?article_id=0000002569&offi...,20210723,비즈니스워치,"\n2030년 65%, 2040년 100% 로드맵 구축국가별 전력소비·환경 분석해 ..."
2,"카뱅·카페 효과는 선반영…카카오, 남은 카드를 보라",/news/news_read.nhn?article_id=0004620923&offi...,20210723,머니투데이,\n\t\t\t\t\t\t\t [[오늘의 포인트]]카카오 로고이달 들어 약세를 이어...
3,"기관, 상승장 배팅 vs ""당분간 박스권""…기로에 선 코스피",/news/news_read.nhn?article_id=0004942370&offi...,20210723,아시아경제,"\n외국인, 23일 다시 순매도기관, 상승장 배팅 레버리지 사들여""미중 고위급 회담..."
4,중국인들이 전기차 사니…요즘 증시서 확 뜬 '이 광물',/news/news_read.nhn?article_id=0004620943&offi...,20210723,머니투데이,"\n\t\t\t\t\t\t\t [7월 들어 중국증시서 리튬 관련주, 2차전지·전기차..."
...,...,...,...,...,...
87,"‘경영권 분쟁’ 헬릭스미스, 소액주주와 화해 무드… 불씨는 남아",/news/news_read.nhn?article_id=0000014715&offi...,20210723,이코노미스트,"\n‘소액주주 추천이사 대표이사 선임’, ‘투자 유치’ 등에 미묘한 입장 엇갈려 바..."
88,"코스피, 4.21P 오른 3254.42 마감(0.13%↑)",/news/news_read.nhn?article_id=0004942436&offi...,20210723,아시아경제,"\n\t\t\t\t\t\t\t 코스피, 4.21P 오른 3254.42 마감(0.1..."
89,"[속보] 코스피, 3254.42 상승 마감… 0.13%↑",/news/news_read.nhn?article_id=0000718058&offi...,20210723,머니S,"\n[속보] 코스피, 3254.42 상승 마감… 0.13%↑\n"
90,[코스피] 4.21p(0.13%) 오른 3254.42 마감,/news/news_read.nhn?article_id=0005497640&offi...,20210723,뉴스1,\n© News1 DB(서울=뉴스1) 전민 기자 = 23일 코스피 마감\n


In [86]:
date[:10]

['20070101',
 '20070102',
 '20070103',
 '20070104',
 '20070105',
 '20070106',
 '20070107',
 '20070108',
 '20070109',
 '20070110']

In [88]:
num_cpu = multiprocessing.cpu_count() - 1
pool = Pool(num_cpu)

In [ ]:
pool.map(F_ftn, date[:10])

In [ ]:
Total = pd.DataFrame({"Title":[],
                        "Href":[],
                        "Date":[],
                        "Press":[]})
for yy in range(2007,2022):
    for mm in range(1,13):
        for dd in range(1,32):
            date = str(yy*10000+mm*100+dd)
            res = req.urlopen("https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page=1".format(date))
            soup = BeautifulSoup(res,"lxml")
            if len(soup.find("ul",class_="newsList").find_all("a")) == 0:
                continue
            try:
                Last_page = int(soup.find("td",class_ = "pgRR").find("a").get("href").split("=")[-1])
                date_total = pd.concat([ftn(date,page) for page in range(1,Last_page+1)]
                                  ,ignore_index=True)
            except:
                date_total = ftn(date,1)
            Total = pd.concat([Total,date_total])
            print(date)